In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random as rnd
import urllib.request
import re
import string
import math
%matplotlib inline

In [32]:
def dataPreProcess(data):
  f = urllib.request.urlopen(data)
  tweets_list = []
  for sentence in f:
    tweets_list.append(sentence.decode('UTF-8'))
  filtered_tweets = []
  for i in range(len(tweets_list)):
    #Removing any '\n' at the end of every sentence
    tweets_list[i] = tweets_list[i].strip('\n')

    #Removing tweet id and time stamp                                                          
    tweets_list[i] = tweets_list[i][50:]   

    #Removing any word that starts with the symbol @                                                          
    tweets_list[i] = " ".join(filter(lambda twt: twt[0] != '@', tweets_list[i].split())) 

    #Removing any hashtag # symbols         
    tweets_list[i] = tweets_list[i].replace('#','')     

    #Removing URLs that begin with http                                              
    tweets_list[i] = re.sub(r"http\S+", "",tweets_list[i])     

    #Removing URLs that begin with www                                       
    tweets_list[i] = re.sub(r"www\S+", "",tweets_list[i])  

    #Converting every word to lower case                                          
    tweets_list[i] = tweets_list[i].lower()

    #Removing any other puntutation marks                                                           
    tweets_list[i] = tweets_list[i].translate(tweets_list[i].maketrans('','',string.punctuation))     

    filtered_tweets.append(tweets_list[i].split())
  f.close()
  print(filtered_tweets)
  return filtered_tweets




In [33]:
def k_Means(tweets, k = 5, maximum_iterations = 100):
  centroids = []
  count = 0
  random_centroid = dict()
  while count < k:
    random_index = rnd.randint(0, len(tweets) - 1)
    if random_index not in random_centroid:
      random_centroid[random_index] = True
      centroids.append(tweets[random_index])
      count += 1

  iteration = 0
  previous_centroid = []

   # Running the iterations until not converged or until the max iteration in not reached
  while(is_converging(previous_centroid, centroids)) == False and (iteration < maximum_iterations):
    #print("running iteration " + str(iteration))
    
    # Assingning tweets to the closest centroids
    clusters = allot_cluster(tweets, centroids)

    #keeping a track if k-means converges 
    previous_centroid = centroids

    # Update centroids based on clusters formed
    centroids = reassign_centroids(clusters)
    iteration += 1

  if iteration == maximum_iterations:
    print("Maximum iterations have been reached. K-Means not converging!")
  
  sse = calculate_SSE(clusters)
  return clusters, sse

In [34]:
def is_converging(previous_centroid, centroids):
  if len(previous_centroid) != len(centroids):
    return False
    
  for i in range(len(centroids)):
    if ' '.join(previous_centroid[i]) != ' '.join(centroids[i]):
      return False
  return True


In [35]:
def allot_cluster(tweets, centroids):
  clusters = dict()
  for i in range(len(tweets)):

    # Initializing minimum_distance to infinite distance intially.
    minimum_distance = math.inf                           
    cluster_index = -1
    for j in range(len(centroids)):
      distance = jaccard_distance(tweets[i], centroids[j])
      if centroids[j] == tweets[i]:
        cluster_index = j
        minimum_distance = 0
        break
      if distance < minimum_distance:
        cluster_index = j
        minimum_distance = distance
    if minimum_distance == 1:                                     # No intersection / nothing common.
      cluster_index = rnd.randint(0, len(centroids) - 1)
    
    clusters.setdefault(cluster_index, []).append([tweets[i]])
    last_tweet_index = len(clusters.setdefault(cluster_index, [])) - 1
    clusters.setdefault(cluster_index, [])[last_tweet_index].append(minimum_distance)
  return clusters

In [36]:
def reassign_centroids(clusters):
  centroids =[]
  # iterate each cluster and check for a tweet with closest distance sum with all other tweets in the same cluster
  # select that tweet as the centroid for the cluster
  for i in range(len(clusters)):
    minimum_distance_sum = math.inf
    centroid_indx = -1
    # To avoid unnecessary calculations 
    minimum_distance_dp = []

    for t1 in range(len(clusters[i])):
      minimum_distance_dp.append([])
      distance_sum = 0
      for t2 in range(len(clusters[i])):
        if t1 != t2:
          if t2 < t1:
            distance = minimum_distance_dp[t2][t1]
          else:
            distance = jaccard_distance(clusters[i][t1][0], clusters[i][t2][0])

          minimum_distance_dp[t1].append(distance)
          distance_sum += distance
        else:
          minimum_distance_dp[t1].append(0)
        
      if distance_sum < minimum_distance_sum:
        minimum_distance_sum = distance_sum
        centroid_indx = t1

    centroids.append(clusters[i][centroid_indx][0])
  return centroids

In [37]:
def is_converging(previous_centroid, centroids):
  if len(previous_centroid) != len(centroids):
    return False
  for i in range(len(centroids)):
    if ' '.join(previous_centroid[i]) != ' '.join(centroids[i]):
      return False
  return True

In [38]:
def jaccard_distance(t1, t2):
  intersection_operation = set(t1).intersection(t2)
  union_operation = set().union(t1, t2)
  return 1 - (len(intersection_operation) / len(union_operation))

In [39]:
def calculate_SSE(clusters):
  sse = 0
  for i in range(len(clusters)):
    for t in range(len(clusters[i])):
      sse = sse + (clusters[i][t][1] * clusters[i][t][1])
  return sse

In [40]:
if __name__ == '__main__':
  data = "https://raw.githubusercontent.com/sirishasatish/K-Means/main/nytimeshealth.txt"
  twt = dataPreProcess(data)
  number_of_k = 8
  k = 3
  for i in range(number_of_k):
    print("For k = " + str(k)+',')

    clusters, sse = k_Means(twt, k)
    for c in range(len(clusters)):
      print('Cluster ' + str(c+1) + " has ", str(len(clusters[c])) + " tweets")
    print("SSE : " + str(sse))
    print()
    k += 1





[['risks', 'in', 'using', 'social', 'media', 'to', 'spot', 'signs', 'of', 'mental', 'distress'], ['rt', 'the', 'most', 'effective', 'nationwide', 'diabetes', 'prevention', 'program', 'youve', 'probably', 'never', 'heard', 'of'], ['the', 'new', 'old', 'age', 'blog', 'diabetes', 'prevention', 'that', 'works'], ['well', 'comfort', 'casseroles', 'for', 'winter', 'dinners'], ['highlevel', 'knowledge', 'before', 'veterans', 'affairs', 'scandal'], ['your', 'money', 'affordable', 'care', 'act’s', 'tax', 'effects', 'now', 'loom', 'for', 'filers'], ['well', 'christmas', 'in', 'the', 'hospital'], ['the', 'texas', 'tribune', 'for', 'members', 'of', 'health', 'ministries', 'in', 'texas', 'caring', 'means', 'sharing', 'the', 'bills'], ['a', 'qampa', 'with', 'laura', 'hillenbrand', 'author', 'of', 'unbroken', 'who', 'suffers', 'from', 'chronic', 'fatigue', 'syndrome'], ['race', 'to', 'deliver', 'nicotine’s', 'punch', 'with', 'less', 'risk'], ['the', 'new', 'health', 'care', 'people', 'are', 'shopping